# **Project - Convolutional Neural Networks: Street View Housing Number Digit Recognition**

# Marks: 30

Welcome to the project on classification using Convolutional Neural Networks. We will continue to work with the Street View Housing Numbers (SVHN) image dataset for this project.

--------------
### **Context:** 
--------------

One of the most interesting tasks in deep learning is to recognize objects in natural scenes. The ability to process visual information using machine learning algorithms can be very useful as demonstrated in various applications.

The SVHN dataset contains over 600,000 labeled digits cropped from street-level photos. It is one of the most popular image recognition datasets. It has been used in neural networks created by Google to improve map quality by automatically transcribing the address numbers from a patch of pixels. The transcribed number with a known street address helps pinpoint the location of the building it represents. 

----------------
### **Objective:**
----------------

Build a CNN model that can identify the digits in the images.

-------------
### **Dataset**
-------------
Here, we will use a subset of the original data to save some computation time. The dataset is provided as a .h5 file. The basic preprocessing steps have been done.

## **Mount the drive**
Let us start by mounting the drive and importing the necessary libraries.

In [7]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Running setup.py clean for pandas
Failed to build pandas
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.17
    Uninstalling prompt-toolkit-3.0.17:
      Successfully uninstalled prompt-toolkit-3.0.17
  Attempting uninstall: ipython
    Found existing installation: ipython 7.

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\hmamo\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\hmamo\\AppData\\Local\\Temp\\pip-install-40ybaid6\\pandas_f7ed72ae721c40dfb0950f108d146e15\\setup.py'"'"'; __file__='"'"'C:\\Users\\hmamo\\AppData\\Local\\Temp\\pip-install-40ybaid6\\pandas_f7ed72ae721c40dfb0950f108d146e15\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\hmamo\AppData\Local\Temp\pip-wheel-w4nuw7vb'
       cwd: C:\Users\hmamo\AppData\Local\Temp\pip-install-40ybaid6\pandas_f7ed72ae721c40dfb0950f108d146e15\
  Complete output (1935 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-3.8\pandas
  copyi

ModuleNotFoundError: No module named 'google.colab'


  copying pandas\tests\reshape\merge\test_merge.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  copying pandas\tests\reshape\merge\test_merge_asof.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  copying pandas\tests\reshape\merge\test_merge_index_as_string.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  copying pandas\tests\reshape\merge\test_merge_ordered.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  copying pandas\tests\reshape\merge\test_multi.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  copying pandas\tests\reshape\merge\__init__.py -> build\lib.win-amd64-3.8\pandas\tests\reshape\merge
  creating build\lib.win-amd64-3.8\pandas\tests\scalar\interval
  copying pandas\tests\scalar\interval\test_interval.py -> build\lib.win-amd64-3.8\pandas\tests\scalar\interval
  copying pandas\tests\scalar\interval\test_ops.py -> build\lib.win-amd64-3.8\pandas\tests\scalar\interval
  copying pandas\tests\scalar\interval\__init__.py -> build

  pandas\_libs/algos.c(44781): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(45644): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(46507): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(47370): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(48233): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(49103): warning C4244: '=': conversion from 'long' to 'char', possible loss of data
  pandas\_libs/algos.c(49967): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(50436): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/algos.c(50906): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of da

  pandas\_libs/groupby.c(11058): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
  pandas\_libs/groupby.c(16680): warning C4244: '=': conversion from 'npy_double' to '__pyx_t_5numpy_float32_t', possible loss of data
  pandas\_libs/groupby.c(17384): warning C4244: '=': conversion from 'npy_double' to '__pyx_t_5numpy_float32_t', possible loss of data
  pandas\_libs/groupby.c(18149): warning C4244: '=': conversion from 'npy_double' to '__pyx_t_5numpy_float32_t', possible loss of data
  pandas\_libs/groupby.c(18171): warning C4244: '/=': conversion from 'double' to '__pyx_t_5numpy_float32_t', possible loss of data
  pandas\_libs/groupby.c(18877): warning C4244: '=': conversion from 'npy_double' to '__pyx_t_5numpy_float32_t', possible loss of data
  pandas\_libs/groupby.c(22280): warning C4244: '=': conversion from 'Py_ssize_t' to '__pyx_t_5numpy_float64_t', possible loss of data
  pandas\_libs/groupby.c(22325): warning C4244: '=': conversion from 'Py_ssize_

    [18/36] Cythonizing pandas\_libs/sparse.pyx
    [19/36] Cythonizing pandas\_libs/testing.pyx
    [20/36] Cythonizing pandas\_libs/tslib.pyx
    [21/36] Cythonizing pandas\_libs/tslibs/ccalendar.pyx
    [22/36] Cythonizing pandas\_libs/tslibs/conversion.pyx
    [23/36] Cythonizing pandas\_libs/tslibs/fields.pyx
    [24/36] Cythonizing pandas\_libs/tslibs/frequencies.pyx
    [25/36] Cythonizing pandas\_libs/tslibs/nattype.pyx
    [26/36] Cythonizing pandas\_libs/tslibs/np_datetime.pyx
    [27/36] Cythonizing pandas\_libs/tslibs/offsets.pyx
    [28/36] Cythonizing pandas\_libs/tslibs/parsing.pyx
    [29/36] Cythonizing pandas\_libs/tslibs/period.pyx


    [30/36] Cythonizing pandas\_libs/tslibs/resolution.pyx
    [31/36] Cythonizing pandas\_libs/tslibs/strptime.pyx
    [32/36] Cythonizing pandas\_libs/tslibs/timedeltas.pyx
    [33/36] Cythonizing pandas\_libs/tslibs/timestamps.pyx
    [34/36] Cythonizing pandas\_libs/tslibs/timezones.pyx
    [35/36] Cythonizing pandas\_libs/writers.pyx
    [36/36] Cythonizing pandas\io/sas/sas.pyx
    running install
    running build
    running build_py
    creating build
    creating build\lib.win-amd64-3.8
    creating build\lib.win-amd64-3.8\pandas
    copying pandas\conftest.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\testing.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\_version.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\__init__.py -> build\lib.win-amd64-3.8\pandas
    creating build\lib.win-amd64-3.8\pandas\api
    copying pandas\api\__init__.py -> build\lib.win-amd64-3.8\pandas\api
    creating build\lib.win-amd64-3.8\pandas\arrays
    copying pandas\arra

    C:\Program Files (x86)\Microsoft Visual Studio\2017\BuildTools\VC\Tools\MSVC\14.16.27023\bin\HostX86\x64\link.exe /nologo /INCREMENTAL:NO /LTCG /DLL /MANIFEST:EMBED,ID=2 /MANIFESTUAC:NO /LIBPATH:C:\Users\hmamo\anaconda3\libs /LIBPATH:C:\Users\hmamo\anaconda3\PCbuild\amd64 "/LIBPATH:C:\Program Files (x86)\Microsoft Visual Studio\2017\BuildTools\VC\Tools\MSVC\14.16.27023\lib\x64" "/LIBPATH:C:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\lib\um\x64" "/LIBPATH:C:\Program Files (x86)\Windows Kits\10\lib\10.0.17763.0\ucrt\x64" "/LIBPATH:C:\Program Files (x86)\Windows Kits\10\lib\10.0.17763.0\um\x64" /EXPORT:PyInit_reshape build\temp.win-amd64-3.8\Release\pandas\_libs/reshape.obj /OUT:build\lib.win-amd64-3.8\pandas\_libs\reshape.cp38-win_amd64.pyd /IMPLIB:build\temp.win-amd64-3.8\Release\pandas\_libs\reshape.cp38-win_amd64.lib
       Creating library build\temp.win-amd64-3.8\Release\pandas\_libs\reshape.cp38-win_amd64.lib and object build\temp.win-amd64-3.8\Release\pandas\_libs\reshape

## **Importing libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten, LeakyReLU
from tensorflow.keras.utils import to_categorical

Let us check for the version of TensorFlow.

In [ ]:
print(tf.__version__)

## **Load the dataset**
- Let us now load the dataset that is available as a .h5 file.
- Split the data into train and the test dataset

In [ ]:
import h5py

# Open the file as read only
# User can make changes in the path as required
h5f = h5py.File('/content/drive/MyDrive/Colab Notebooks/SVHN_single_grey1.h5', 'r')

# Load the training and the test set
X_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
X_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

Let's check the number of images in the training and testing data.

In [ ]:
len(X_train), len(X_test)

**Observations**
- There are 42,000 images in the training data and 18,000 images in the testing data. 

## **Visualizing images**
- Use X_train to visualize the first 10 images
- Use Y_train to print the first 10 labels

#### **Question 1: Complete the below code to visualize the first 10 images from the training data (1 Mark)**

In [ ]:
# visualizing the first 10 images in the dataset and their labels
plt.figure(figsize=(10, 1))

for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X_train[i], cmap="gray") # write the function to visualize images
    plt.axis('off')

plt.show()
print('label for each of the above image: %s' % (y_train[0:10]))

## **Data Preparation**

- Print the first image in the train image and figure out the shape of the images
- Reshape the train and the test dataset to make them fit the first convolutional operation that we will create later. Figure out the required shape
- Normalize the train and the test dataset by dividing by 255
- Print the new shapes of the train and the test set
- One-hot encode the target variable

In [ ]:
# Shape of the images and the first image

print("Shape:", X_train[0].shape)
print()
print("First image:\n", X_train[0])

In [ ]:
# Reshaping the dataset to flatten them. Remember that we always have to give a 4D array as input to CNNs

X_train = X_train.reshape(X_train.shape[0], 32,32,1)
X_test = X_test.reshape(X_test.shape[0], 32,32,1)

In [ ]:
# Normalize inputs from 0-255 to 0-1

X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# New shape 

print('Training set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

#### **Question 2: One-hot encode the labels in the target variable y_train and y_test (2 Marks)**

In [ ]:
#Write the function and appropriate variable name to one-hot encode the output

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
#test labels
y_test

- Notice that each entry of y_test is a one-hot encoded vector instead of a single label.

## **Model Building**

Now, we have done data preprocessing, let's build a CNN model.

In [ ]:
#Fixing the seed for random number generators
np.random.seed(42)
import random
random.seed(42)
tf.random.set_seed(42)

### **Model Architecture**
- Write a function that returns a sequential model with the following architecture
 - First Convolutional layer with **16 filters and kernel size of 3x3**. Use the **'same' padding** and provide an **apt input shape**
 - Add a **LeakyRelu layer** with the **slope equal to 0.1**
 - Second Convolutional layer with **32 filters and kernel size of 3x3 with 'same' padding**
 - Another **LeakyRelu** with the **slope equal to 0.1**
 - A **max-pooling layer** with a **pool size of 2x2**
 - **Flatten** the output from the previous layer
 - Add a **dense layer with 32 nodes**
 - Add a **LeakyRelu layer with slope equal to 0.1**
 - Add the final **output layer with nodes equal to the number of classes** and **softmax activation**
 - Compile the model with the **categorical_crossentropy loss, adam optimizers (learning_rate = 0.001), and accuracy metric**. Do not fit the model here, just return the compiled model
- Call the function and store the model in a new variable
- Print the summary of the model.
- Fit the model on the train data with a **validation split of 0.2, batch size = 32, verbose = 1, and 20 epochs**. Store the model building history to use later for visualization.

#### **Question 3: Build and train a CNN model as per the above mentioned architecture (10 Marks)**

In [ ]:
#Importing losses and optimizers modules
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

#Define the function
def cnn_model_1():
    model = Sequential() 
    #Add layers as per the architecture mentioned above in the same sequence
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last', input_shape=(32,32,1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(10, activation='softmax'))

    #declare adam optimizer with learning rate of 0.001 
    adam = optimizers.Adam(learning_rate=0.001)
    
    #compile the model
    model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [ ]:
#Build the model
model_1 = cnn_model_1()

In [ ]:
#Print the model summary
model_1.summary()

In [ ]:
#Fit the model on the train data with a validation split of 0.2, batch size = 32, verbose = 1, and 20 epochs. Store the model building history
# Fit the model
history_model_1 = model_1.fit(X_train, y_train, validation_data=(X_test, y_test), verbose = 1, batch_size=32, epochs = 20)

### **Plotting the validation and training accuracies**

#### **Question 4: Write your observations on the below plot (2 Marks)**

In [ ]:
# plotting the accuracies

dict_hist = history_model_1.history
list_ep = [i for i in range(1,21)]

plt.figure(figsize = (8,8))
plt.plot(list_ep,dict_hist['accuracy'],ls = '--', label = 'accuracy')
plt.plot(list_ep,dict_hist['val_accuracy'],ls = '--', label = 'val_accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

**Observations:__________**

- With the current architecture the model is with 85% accuracy on the training 267,306 parameters 
- but not on the validation set 

Let's build another model and see if we can get a better model with generalized performance.

First, we need to clear the previous model's history from the keras backend. Also, let's fix the seed again after clearing the backend.

In [ ]:
#Clearing backend
from tensorflow.keras import backend
backend.clear_session()

In [ ]:
#Fixing the seed for random number generators
np.random.seed(42)
import random
random.seed(42)
tf.random.set_seed(42)

### **Second Model Architecture**
- Write a function that returns a sequential model with the following architecture
 - First Convolutional layer with **16 filters and kernel size of 3x3**. Use the **'same' padding** and provide an **apt input shape**
 - Add a **LeakyRelu layer** with the **slope equal to 0.1**
 - Second Convolutional layer with **32 filters and kernel size of 3x3 with 'same' padding**
 - Add **LeakyRelu** with the **slope equal to 0.1**
 - Add a **max-pooling layer** with a **pool size of 2x2**
 - Add a **BatchNormalization layer**
 - Third Convolutional layer with **32 filters and kernel size of 3x3 with 'same' padding**
 - Add a **LeakyRelu layer with slope equal to 0.1**
 - Fourth Convolutional layer **64 filters and kernel size of 3x3 with 'same' padding** 
 - Add a **LeakyRelu layer with slope equal to 0.1**
 - Add a **max-pooling layer** with a **pool size of 2x2**
 - Add a **BatchNormalization layer**
 - **Flatten** the output from the previous layer
 - Add a **dense layer with 32 nodes**
 - Add a **LeakyRelu layer with slope equal to 0.1**
 - Add a **dropout layer with rate equal to 0.5**
 - Add the final **output layer with nodes equal to the number of classes** and **softmax activation**
 - Compile the model with the **categorical_crossentropy loss, adam optimizers (learning_rate = 0.001), and accuracy metric**. Do not fit the model here, just return the compiled model
- Call the function and store the model in a new variable
- Print the summary of the model.
- Fit the model on the train data with a **validation split of 0.2, batch size = 128, verbose = 1, and 30 epochs**. Store the model building history to use later for visualization.

#### **Question 5: Build and train the second CNN model as per the above mentioned architecture (10 Marks)**

In [ ]:
#Define the function
def cnn_model_2():
    model = Sequential() 
    #Add layers as per the architecture mentioned above in the same sequence
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last', input_shape=(32,32,1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())


    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(rate=0.5))

    model.add(Dense(10, activation='softmax'))    




    #declare adam optimizer with learning rate of 0.001 
    adam = optimizers.Adam(learning_rate=0.001)
    
    #compile the model
    model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [ ]:
# Build the model
model_2 = cnn_model_2()

In [ ]:
#Print the summary
model_2.summary()

In [ ]:
# Fit the model
#Fit the model on the train data with a validation split of 0.2, batch size = 128, verbose = 1, and 30 epochs. Store the model building
history_model_2 = model_2.fit(X_train, y_train, validation_data=(X_test, y_test), verbose = 1, batch_size=128, epochs = 30)

### **Plotting the validation and training accuracies**

#### **Question 6: Write your observations on the below plot (2 Marks)**

In [ ]:
# plotting the accuracies

dict_hist = history_model_2.history
list_ep = [i for i in range(1,31)]

plt.figure(figsize = (8,8))
plt.plot(list_ep,dict_hist['accuracy'],ls = '--', label = 'accuracy')
plt.plot(list_ep,dict_hist['val_accuracy'],ls = '--', label = 'val_accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

**Observations:________**

By training 164,170 param, the model could reach an accuracy 0.9127 on training data, and almost similar on the validation data


## **Predictions on the test data**

- Make predictions on the test set using the second model
- Print the obtained results using the classification report and the confusion matrix
- Final observations from the obtained results

#### **Question 7: Make predictions on the test data using the second model (1 Mark)**

In [ ]:
#Make prediction on the test data using model_2 
test_pred = model_2.predict(X_test)

test_pred = np.argmax(test_pred, axis=-1)

**Note:** Earlier, we noticed that each entry of the test data is a one-hot encoded vector but to print the classification report and confusion matrix, we must convert each entry of y_test to a single label.

In [ ]:
#Converting each entry to single label from one-hot encoded vector
y_test = np.argmax(y_test, axis=-1)

#### **Question 8: Write your final observations on the performance of the model on the test data (2 Marks)**

In [ ]:
#importing required functions
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#Printing the classification report
print(classification_report(y_test, test_pred))

#Plotting the heatmap using confusion matrix
cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(8,5))
sns.heatmap(cm, annot=True,  fmt='.0f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#### **Observations:_________**